### 상대모멘텀 
- 10개의 주식 데이터를 이용하여 투자 전략 테스팅
1. 월초부터 월말까지의 수정주가를 이용하여 월별 수익율
2. 월별 수익율이 높은 순서중 n개의 주식을 선택 
3. 해당하는 주식들을 매수 후 매도하여 수익율 계산

In [ ]:
import pandas as pd 
from datetime import datetime
import numpy as np
import os 
import glob
import warnings

In [ ]:
# warning message 제거 
warnings.filterwarnings('ignore')

In [ ]:
# 월별 수익율을 계산하는 함수 생성 
def create_1M_rtn(_df, _ticker, _start = "2010-01-01", _col = 'Adj Close'):
    # 복사본 생성 
    result = _df.copy()
    # 컬럼에 Date가 포함되어있는지 확인 
    if 'Date' in result.columns:
        result = result.loc[result['Date'] >= _start, ['Date', _col]]
        # Date 컬럼의 데이터를 시계열로 변경
        result['Date'] = pd.to_datetime(result['Date'], format='%Y-%m-%d')
        result.set_index('Date', inplace=True)
    else:
        result.index = pd.to_datetime(result.index, inplace=True)
        result = result.loc[_start:, [_col]]
    # 기준 년월 컬럼을 생성
    result['STD-YM'] = result.index.strftime('%Y-%m')
    result['1m_rtn'] = 0
    result['CODE'] = _ticker
    # 기준 년월의 중복데이터를 제거하고 고유한 값들을 리스트로 생성
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [ ]:
aapl = pd.read_csv('../../csv/AAPL.csv')

In [ ]:
sample_aapl, ym_list = create_1M_rtn(aapl, 'AAPL')

In [ ]:
sample_aapl.head()

In [ ]:
ym_list

In [ ]:
# 특정 경로에 있는 파일들의 목록을 로드 
os.listdir("./data")

files = glob.glob("./data/*.csv")

# 새로운 데이터프레임을 생성 
# 종목별 데이터프레임 
stock_df = pd.DataFrame()
# 월말 데이터프레임 
month_last_df = pd.DataFrame()

for file in files:
    folder, name = os.path.split(file)
    # print(folder, name)
    head, tail = os.path.splitext(name)
    # print(head, tail)
    # head는 create_1M_rtn 함수에 ticker 인자값으로 사용

    # 데이터 파일을 로드 
    read_df = pd.read_csv(file)

    # create_1M_rtn 함수를 호출 
    price_df, ym_list = create_1M_rtn(read_df, head)

    # 유니언 결합 (단순한 행 결합 함수)
    stock_df = pd.concat([stock_df, price_df], axis=0)

    # 월별 상태 모멘텀을 계산하기 위해 1개월간의 수익율 계산
    for ym in ym_list:
        flag = price_df['STD-YM'] == ym
        m_rtn = price_df.loc[flag,].iloc[-1, 0] / price_df.loc[flag,].iloc[0, 0]
        price_df.loc[flag, '1m_rtn'] = m_rtn
        data = price_df.loc[flag, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat([month_last_df, data], axis=0)

In [ ]:
month_last_df.head()

In [ ]:
month_rtn_df = month_last_df.copy()

In [ ]:
month_rtn_df.reset_index(inplace=True)

In [ ]:
month_rtn_df = month_rtn_df.pivot_table(
    index = 'Date', 
    columns= 'CODE', 
    values= '1m_rtn'
)

In [ ]:
month_rtn_df = month_rtn_df.rank(
    axis=1, 
    ascending=False, 
    method='max', 
    pct=True
)

In [ ]:
# 상위 15% 종목만 선택 
# where(조건식, 거짓일때 변경될 데이터)
month_rtn_df =  month_rtn_df.where(month_rtn_df < 0.15, 0)

In [ ]:
# 데이터 중에서 0이 아닌 데이터는 1로 변경 
month_rtn_df[month_rtn_df != 0] = 1

In [ ]:
month_rtn_df

In [ ]:
# CODE 값들을 유니크 데이터만 따로 출력 
stock_codes = stock_df['CODE'].unique()
stock_codes

In [ ]:
month_rtn_df.loc["2010-01-29 00:00:00", month_rtn_df.iloc[0] >= 1].index

In [ ]:
sig_dict = dict()

for date in month_rtn_df.index:
    # print(date)
    ticker_list = list(
        month_rtn_df.loc[date, month_rtn_df.loc[date] >= 1].index)
    # print(ticker_list)
    sig_dict[date] = ticker_list

In [ ]:
sig_dict

In [ ]:
# 거래 내역컬럼을 생성하는 데이터프레임 생성하는 함수 
def create_trade_book(_df, _code):
    book = _df[_code].copy()
    book['STD-YM'] = book.index.strftime('%Y-%m')
    for c in _code:
        book['p'+c] = ""
        book['r'+c] = ""
    return book

In [ ]:
stock_df.head()

In [ ]:
stock_c_matrix = stock_df.reset_index().pivot_table(
    index='Date', 
    columns = 'CODE', 
    values= stock_df.columns[0]
)

In [ ]:
stock_c_matrix

In [ ]:
book = create_trade_book(stock_c_matrix, stock_codes)

In [ ]:
book.columns

In [71]:
# 포지션을 생성 
for date, values in sig_dict.items():
    # print(date, values)
    for stock in values:
        book.loc[date, 'p'+stock] = 'ready'+stock

2010-01-29 00:00:00 ['BND']
2010-02-26 00:00:00 ['AAPL']
2010-03-31 00:00:00 ['AAPL']
2010-04-30 00:00:00 ['AAPL']
2010-05-28 00:00:00 ['GLD']
2010-06-30 00:00:00 ['GDX']
2010-07-30 00:00:00 ['USM']
2010-08-31 00:00:00 ['GDX']
2010-09-30 00:00:00 ['AMZN']
2010-10-29 00:00:00 ['SLV']
2010-11-30 00:00:00 ['SLV']
2010-12-31 00:00:00 ['SLV']
2011-01-31 00:00:00 ['AAPL']
2011-02-28 00:00:00 ['SLV']
2011-03-31 00:00:00 ['SLV']
2011-04-29 00:00:00 ['SLV']
2011-05-31 00:00:00 ['BND']
2011-06-30 00:00:00 ['MSFT']
2011-07-29 00:00:00 ['SLV']
2011-08-31 00:00:00 ['GLD']
2011-09-30 00:00:00 ['AMZN']
2011-10-31 00:00:00 ['GM']
2011-11-30 00:00:00 ['USM']
2011-12-30 00:00:00 ['AAPL']
2012-01-31 00:00:00 ['GM']
2012-02-29 00:00:00 ['AAPL']
2012-03-30 00:00:00 ['AMZN']
2012-04-30 00:00:00 ['AMZN']
2012-05-31 00:00:00 ['BND']
2012-06-29 00:00:00 ['AMZN']
2012-07-31 00:00:00 ['AAPL']
2012-08-31 00:00:00 ['SLV']
2012-09-28 00:00:00 ['GDX']
2012-10-31 00:00:00 ['GM']
2012-11-30 00:00:00 ['AMZN']
2012-12-3

In [73]:
book[['pBND']].iloc[15:25]

CODE,pBND
Date,
2010-01-26,
2010-01-27,
2010-01-28,
2010-01-29,readyBND
2010-02-01,
2010-02-02,
2010-02-03,
2010-02-04,
2010-02-05,


In [74]:
# 거래 내역 추가 
def trading(_book, s_code):
    std_ym = ""
    buy_phase = False

    # 종목별로 순회
    for code in s_code:
        for i in _book.index:
            # 해당 종목코드의 포지션을 잡는다.
            if (_book.loc[i, 'p'+code] == "") & \
                (_book.shift().loc[i, 'p'+code] == "ready"+code):
                std_ym = book.loc[i, 'STD-YM']
                buy_phase = True
            # 해당 종목코드에서 신호가 잡혀있다면 매수 상태 유지 
            if (_book.loc[i, 'p'+code] == "") & \
                (_book.loc[i, 'STD-YM'] == std_ym) & \
                (buy_phase):
                _book.loc[i, 'p'+code] = 'buy'+code
            
            # std_ym, buy_phase 초기화
            if book.loc[i, 'p'+code] == "":
                std_ym = ""
                buy_phase = False
    return book
                

In [75]:
book = trading(book, stock_codes)

In [76]:
book['pAAPL'].value_counts()

pAAPL
             2078
buyAAPL       292
readyAAPL      14
Name: count, dtype: int64